In [1]:
from pathlib import Path
import frads as fr
from frads import methods, parsers

## Read the configuration file

In [2]:
config = parsers.parse_mrad_config(Path("three_phase.cfg"))

## Assemble the model and generate the matrices
Note: the "no_multiply" option is set to True in the configuration file

In [3]:
with methods.assemble_model(config) as model:
    mtx_paths = methods.three_phase(model, config)

## Load view and daylight matrices into memory

In [16]:
matrices = {'vmx': {}, 'dmx': {}}
for i in range(1, 51):
    matrices['vmx'][i] = fr.load_matrix(mtx_paths.pvmx[f's_floors_glazing.{i}'])
    matrices['dmx'][i] = fr.load_matrix(mtx_paths.dmx[f's_glazing.{i}'])

## Load blinds BSDF into memory

In [29]:
bsdf = {}
for i in range(0, 90, 10):
    bsdf[i] = fr.load_matrix(f"Resources/BSDF/blinds{i:02d}.xml")

## Loop through the weather file and implement control

In [22]:
epw_path = "Resources/NLD_Amsterdam.062400_IWEC.epw"
with open(epw_path) as f:
    meta, wea = parsers.parse_epw(f.read())

In [34]:
window_wpis = []
for w in weas:
    sky_mtx = fr.load_matrix(fr.genskymtx(wea[:1], meta, mfactor=4))
    # We decide how to controls blinds here
    for wi in range(1, 49):
        window_wpis.append(fr.multiply_rgb(matrices['vmx'][wi], bsdf[0], matrices['dmx'][wi], sky_mtx))
        # compute DGP
        


In [ ]:
# South
# 1 25 26 27 28 29 30 31 32
# North
# 2  3  4 21 22 23 24 45 46
# East
# 12 13 14 33 34 36 37 38 39 40 41 42 43 44 47
# West
# 5  6  7  8  9  10 11 15 16 17 18 19 20 35 48
 